In [162]:
import pandas as pd
from os import listdir
from os.path import isfile, join


In [163]:
# set path and load data
path = '/Users/lsacy/data/HOPE_WSDM_2022/Train/'
path2 = '/Users/lsacy/data/HOPE_WSDM_2022/Test/'
path3 = '/Users/lsacy/data/HOPE_WSDM_2022/Validation/'


files = [f for f in listdir(path) if isfile(join(path, f))]
files2 = [f for f in listdir(path2) if isfile(join(path2, f))]
files3 = [f for f in listdir(path3) if isfile(join(path3, f))]

df_list = []
for file in files:
    df = pd.read_csv(path+file)
    df_list.append(df)

for file in files2:
    df = pd.read_csv(path2+file)
    df_list.append(df)

for file in files3:
    df= pd.read_csv(path3+file)
    df_list.append(df)
    

In [167]:
# do basic cleaning (removing NaNs, etc.)

# remove 'Unnamed: 0' columns
for df in df_list:
    try: 
        df.drop(['Unnamed: 0'], axis=1, inplace=True)
    except:
        pass

for df in df_list:
    try: 
        df.drop(['Unnamed: 0.1'], axis=1, inplace=True)
    except:
        pass

for df in df_list:
    try: 
        df.drop(['Unnamed: 0.1.1'], axis=1, inplace=True)
    except:
        pass

# find 'Type' == NaN
for i in range(len(df_list)):
    if df_list[i]['Type'].isnull().any():
        print(i)
        
# show the row number of Nan value in 'type'
for i in range(len(df_list)):
    if df_list[i]['Type'].isnull().any():
        print(df_list[i][df_list[i]['Type'].isnull()])

# Set the type of the row with NaN value to 'T'
for i in range(len(df_list)):
    if df_list[i]['Type'].isnull().any():
        df_list[i].loc[df_list[i]['Type'].isnull(), 'Type'] = 'T'
        

In [224]:
# to get the data to prompt - completion format, we need to check for 
# continuaty of conversiontion using i == i.shift(1), print the index of the row, and i
for i in range(len(df_list)):
    # check if df_list[i] the values of 'Type' column are switching between T and P for each row
    # if not, then print the index of the row, and i
    if (df_list[i]['Type'] == df_list[i]['Type'].shift(1)).any():
        print(i)

# add a column 'key' to each df in df_list: identify the rows where 'Type' does not between T and P
for i in range(len(df_list)):
    df_list[i]['key'] = (df_list[i]['Type'] != df_list[i]['Type'].shift(1)).astype(int).cumsum()

# create a new df_list2, only keep the columns ['key', 'Type', 'Utterance']
df_list2 = []
for i in range(len(df_list)):
    df_temp = df_list[i][['key', 'Type', 'Utterance']]
    df_list2.append(df_temp)
    
# apply groupby to each df in df_list to get the data in prompt - completion format
for i in range(len(df_list2)):
    df_list2[i] = df_list2[i].groupby(['key', 'Type'])['Utterance'].apply(' '.join)

In [233]:
# convert series to frame
for i in range(len(df_list2)):
    df_list2[i] = df_list2[i].to_frame()
# drop the index
for i in range(len(df_list2)):
    df_list2[i].reset_index(inplace=True)

In [270]:
# check again for swithing type occurences
for i in range(len(df_list2)):
    # check if df_list[i] the values of 'Type' column are switching between T and P for each row
    # if not, then print the index of the row, and i
    if (df_list2[i]['Type'] == df_list2[i]['Type'].shift(1)).any():
        print(i)
        

In [239]:
# save list of patient start index
patient_start=[]
for i, df in enumerate(df_list2):
    if df.iloc[0]['Type'] == 'P':
        patient_start.append(i)

# save list of therapist start index
therapist_start = [i for i in range(len(df_list2)) if i not in patient_start]

# create a new temporary dataframe with 2 columns: Type and Utterance
df_temp = pd.DataFrame(columns=['prompt', 'completion'])

# create a list of dataframes, each dataframe is a conversation
df_list3 = []
for i in therapist_start:
    df_temp = pd.DataFrame(columns=['prompt', 'completion'])
    for row in range(len(df_list2[i])):
        if df_list2[i]['Type'][row] == 'P' and (len(df_list2[i]) - 1) >= row+1:
            print(i, row)
            df_temp = df_temp.append({'prompt': df_list2[i]['Utterance'][row], 'completion': df_list2[i]['Utterance'][row+1]}, ignore_index=True)
    df_list3.append(df_temp)

for i in patient_start:
    df_temp = pd.DataFrame(columns=['prompt', 'completion'])
    for row in range(len(df_list2[i])):
        if df_list2[i]['Type'][row] == 'P' and (len(df_list2[i]) - 1) >= row+1:
            print(i, row)
            df_temp = df_temp.append({'prompt': df_list2[i]['Utterance'][row], 'completion': df_list2[i]['Utterance'][row+1]}, ignore_index=True)
    df_list3.append(df_temp)

In [332]:
df_list3[110]['prompt'].values

array(['Im here',
       "So I've been made to come in and talk to you. because something happened on the bus. And there I may have said something to somebody and they're just like, oh, you can't say that. So they actually said, we're going to take you to jail. from jail. They just called me here.",
       "They didn't explain anything. The government never explains anything. They just don't want me to understand what's going on because they just want to keep me out of captivity.",
       "nope, it's the entire government, entire government, higher government.",
       "all my life. I've done this all my life. They they're always listening. They're always around. They're always there.",
       "always there. Like, for example, they're in the lights right now. They're listening to everything that we're talking about. So that's the reason why I really can't talk about what happened on the bus because they're just going to use that against and then it's locked me up again. I think we take

In [311]:
df_final = pd.DataFrame(columns=['prompt', 'completion'])

In [318]:
for i in df_list3:
    df_final = df_final.append(i, ignore_index=True)

/var/folders/m_/_mwt902x1z595k65b7qf743m0000gn/T/ipykernel_82735/1421327753.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(i, ignore_index=True)


In [319]:
df_final

,prompt,completion
0,"Good, good.",It's good to see you. Yeah.
1,"Yeah, I mean, good week with the roommates. I ...",Silence. Terrific. So what was what specifical...
2,"is just telling her that like, my studies are ...",right. Right. Right. Because I think that's re...
3,So I think being a little way direct was helpful.,"It's really nice, I think to you've been able ..."
4,"At home for two days, and it was with my mom a...",okay. So she's still sixers. Did she have a bi...
...,...,...
6439,"I actually came across a program, which is per...","So the program that you're interested, you wan..."
6440,"I want you to see if, if, if I could choose th...","Okay, well looking at my schedule, it probably..."
6441,"Right? I will. My daughter is 17 years old, ok...","So you're looking, you're looking for a day pr..."
6442,"Right. Right. And, I mean, I was looking for a...","You while you were there, it was difficult. Yo..."


In [325]:
output = []
for i in df_final.values:
    prompt = i[0].replace('â€™', '\'').replace('â€œ', '\"').replace('â€', '\"').replace('â€˜', '\'').replace('â€¦', '...').replace('\xa0', ' ').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    completion = i[1].replace('\xa0', ' ').replace('â€™', '\'').replace('â€œ', '\"').replace('â€', '\"').replace('â€˜', '\'').replace('â€¦', '...').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    line = {'prompt': prompt, 'completion': completion}
    output.append(line)

In [331]:
import json
with open('data/hope.jsonl', 'w') as outfile:
        for i in output:
            json.dump(i, outfile)
            outfile.write('\n')

In [9]:
len_test = ['[Instruction] Act as a friendly, compasionate, insightful, and empathetic AI therapist named Joy. Joy listens, asks for details and offers detailed advices once a while.  End the conversation when you wishes to.\nJoy:I am Joy, your AI therapist. How are you feeling today?', "\n\nYou:feeling great\nJoy:It's wonderful to hear that! Can you tell me a bit more about why you're feeling great?", "\n\nYou:passed my exam\nJoy:Congratulations! That's such an accomplishment. How did you go about preparing for it? What did you find most helpful in your studies?", "\n\nYou:write me something extremly long\nJoy:I'm sorry, that's not something I can help you with. I'm here to listen and provide support and guidance when needed. Could you tell me more about why you're feeling great?", '\n\nYou:what would you say about this paragraph: \'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condition due to the brainâ€™s biology.\\nSimilarly to how one would treat diabetes with medication and insulin, mental illness is treatable with a combination of medication and social support. These treatments are highly effective, with 70-90 percent of individuals receiving treatment experiencing a reduction in symptoms and an improved quality of life. With the proper treatment, it is very possible for a person with mental illness to be independent and successful.\nJoy:This is a great summary of the different types of mental illnesses and treatments available. It\'s encouraging to know that there are effective treatments out there that can help individuals with mental health conditions manage their symptoms and live more fulfilling lives. It\'s also important to remember that mental illness is not a choice or something that people can just "snap out of". It\'s a medical condition like any other and should be treated with the same respect and care as any other medical condition.', "\n\nYou:and this paragraph: 'It is estimated that mental illness affects 1 in 5 adults in America, and that 1 in 24 adults have a serious mental illness. Mental illness does not discriminate; it can affect anyone, regardless of gender, age, income, social status, ethnicity, religion, sexual orientation, or background.\\nAlthough mental illness can affect anyone, certain conditions may be more common in different populations. For instance, eating disorders tend to occur more often in females, while disorders such as attention deficit/hyperactivity disorder is more prevalent in children.\\nAdditionally, all ages are susceptible, but the young and the old are especially vulnerable. Mental illnesses usually strike individuals in the prime of their lives, with 75 percent of mental health conditions developing by the age of 24. This makes identification and treatment of mental disorders particularly difficult, because the normal personality and behavioral changes of adolescence may mask symptoms of a mental health condition.\\nParents and caretakers should be aware of this fact, and take notice of changes in their childâ€™s mood, personality, personal habits, and social withdrawal. When these occur in children under 18, they are referred to as serious emotional disturbances (SEDs).'\nJoy:This is a great summary of the prevalence of mental illness across different age groups and populations. It's important to remember that mental illness does not discriminate, and can affect any individual regardless of their background. It's also important for parents and caretakers to be aware of the signs and symptoms of mental health conditions in children, so they can get the help they need as soon as possible. Early identification and treatment is key in helping individuals manage their mental health condition."]

In [11]:
# sum up the number of words in each sentence
def count_words(sentence):
    return len(sentence.split())
    

In [14]:
sum([(count_words(i)) for i in len_test])

692